In [12]:
import boto3
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'omar-vargas-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [13]:
## Defining input and target 
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [14]:
X_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3348.000000,3376.000000,3389.000000
mean,49.660472,0.493215,236.356332,25.773871,75.776335
std,8.634512,0.500028,44.485204,4.033353,11.950464
min,32.000000,0.000000,107.000000,15.960000,44.000000
25%,42.000000,0.000000,205.000000,23.060000,68.000000
50%,49.000000,0.000000,233.000000,25.380000,75.000000
75%,57.000000,1.000000,263.000000,28.007500,82.000000
max,70.000000,1.000000,696.000000,56.800000,140.000000


In [15]:
X_test.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,840.000000,843.000000,848.000000
mean,49.283019,0.497642,238.177381,25.914686,76.288915
std,8.316360,0.500290,45.004126,4.262842,12.324816
min,33.000000,0.000000,143.000000,15.540000,47.000000
25%,42.000000,0.000000,208.000000,23.090000,68.000000
50%,49.000000,0.000000,235.000000,25.410000,75.000000
75%,56.000000,1.000000,262.250000,28.140000,85.000000
max,70.000000,1.000000,453.000000,45.800000,143.000000


In [16]:
## Defining the imputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer.fit(X_train)

## Imputing thr missing values
X_train_imp = pd.DataFrame(imputer.fit_transform(X_train), columns = X_train.columns)
X_test_imp = pd.DataFrame(imputer.fit_transform(X_test), columns = X_test.columns)

In [17]:
X_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.000000,3390.000000,3390.000000
mean,49.660472,0.493215,236.314749,25.772245,75.776106
std,8.634512,0.500028,44.210250,4.025093,11.948708
min,32.000000,0.000000,107.000000,15.960000,44.000000
25%,42.000000,0.000000,206.000000,23.070000,68.000000
50%,49.000000,0.000000,233.000000,25.380000,75.000000
75%,57.000000,1.000000,262.750000,27.990000,82.000000
max,70.000000,1.000000,696.000000,56.800000,140.000000


In [18]:
X_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.283019,0.497642,238.147406,25.911710,76.288915
std,8.316360,0.500290,44.792142,4.250417,12.324816
min,33.000000,0.000000,143.000000,15.540000,47.000000
25%,42.000000,0.000000,208.000000,23.090000,68.000000
50%,49.000000,0.000000,235.000000,25.410000,75.000000
75%,56.000000,1.000000,262.000000,28.100000,85.000000
max,70.000000,1.000000,453.000000,45.800000,143.000000


In [20]:
scaler = MinMaxScaler()

X_train_imp = pd.DataFrame(scaler.fit_transform(X_train_imp), columns = X_train_imp.columns)
X_test_imp = pd.DataFrame(scaler.fit_transform(X_test_imp), columns = X_test_imp.columns)

Logisitc Regession

In [21]:
logit_md = LogisticRegression().fit(X_train_imp, Y_train)

## Predicting on the test
logit_pred = logit_md.predict_proba(X_test_imp)[:,1]

## Changing likelihood to labels
logit_label = np.where(logit_pred < 0.1, 0, 1)

print(classification_report(Y_test, logit_label))

              precision    recall  f1-score   support

           0       0.92      0.29      0.44       719
           1       0.18      0.85      0.29       129

    accuracy                           0.37       848
   macro avg       0.55      0.57      0.36       848
weighted avg       0.80      0.37      0.41       848



Random Forest

In [22]:
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_imp, Y_train)

## Predicting on the test
RF_pred = RF_md.predict_proba(X_test_imp)[:,1]

## Changing likelihood to labels
RF_label = np.where(RF_pred < 0.1, 0, 1)

print(classification_report(Y_test, RF_label))

              precision    recall  f1-score   support

           0       0.92      0.23      0.36       719
           1       0.17      0.89      0.29       129

    accuracy                           0.33       848
   macro avg       0.55      0.56      0.32       848
weighted avg       0.81      0.33      0.35       848



In [ ]:
# Based on my results, I would use the logistic regression model to predict likelihood of TenYearCHD.